# Importações

In [ ]:
# Instala as dependências
!pip install -r ../../../requirements.txt

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

In [ ]:
# Evitando logs no plot de gráficos\n",
pd.options.mode.chained_assignment = None

# Leitura dos dados_tratados

In [ ]:
dados_tratados = pd.read_csv('../../../data/tratado/dados_cvm_tratados.csv')
dados_tratados.info()

In [ ]:
dados_tratados['Razao_Inadimplencia_Provisao'] = dados_tratados['Inadimplencia_Total'] / dados_tratados['Provisao_Total']

# Contextualização
Seguindo o mesmo raciocínio explicitado no *notebook* referente ao modelo *K-Means*, as seções abaixo se destinam à construção de um modelo de clusterização utilizando o algoritmo *DBSCAN* para servir de comparação à modelagem realizada com o algoritmo *Isolation Forest*. Também foi feita a modelagem com a separação dos fundos por carteira e com as mesmas *features* utilizadas no *notebook* referente ao modelo *Isolation Forest*.

O *DBSCAN* é um algoritmo de agupamento popular por se diferenciar dos sistemas mais comuns, como o próprio *K-Means*, em seus processos de clusterização: ele define arbitrariamente a quantidade de *clusters* resultantes e identifica *clusters* de diferentes formatos e densidades de pontos com certa facilidade. Além disso, o *DBSCAN* é eficiente computacionalmente e lida bem com ruído, ou seja, é capaz de identificar e isolar *outliers*. Portanto, esse algoritmo pode ser uma excelente escolha, especialmente quando não se sabe previamente quantos *clusters* esperar e quando os *clusters* têm formas irregulares e densidades variáveis, que pode ser o caso do nosso projeto, considerando o caráter caótico dos dados que possuímos.

# Separação dos fundos por carteira

Utilizando o algoritmo de LabelEncoding, foi criada a coluna "Carteira_Classificacao_Encoded", que apresenta números que identificam fundos de diversas carteiras. Nesse contexto, as labels numéricas criadas são definidas da seguinte forma:

Setor Público = 0

Agronegócio = 1

Cartão = 2

Comercial = 3

Imobiliário = 4

Financeiro = 5

Industrial = 6

Factoring = 7

Multimercado = 8

Serviços = 9

Ações Judiciais = 10

In [ ]:
# Lista de carteiras com índices correspondentes à classificação
classificacao_encoding = [
    'SetorPublico',
    'Agronegocio',
    'Cartao',
    'Comercial',
    'Imobiliario',
    'Financeiro',
    'Industrial',
    'Factoring',
    'Multimercado',
    'Servicos',
    'AcoesJudiciais'
]

In [ ]:
# Separando os dados por carteira
dados_classificacao = {
    'Setor Público': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('SetorPublico')],
    'Agronegócio': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Agronegocio')],
    'Cartão': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Cartao')],
    'Comercial': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Comercial')],
    'Imobiliário': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Imobiliario')],
    'Financeiro': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Financeiro')],
    'Industrial': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Industrial')],
    'Factoring': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Factoring')],
    'Multimercado': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Multimercado')],
    'Serviços': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Servicos')],
    'Ações Judiciais': dados_tratados.loc[dados_tratados['Carteira_Classificação_encoded'] == classificacao_encoding.index('AcoesJudiciais')]
}

# Começo da modelagem com *DBSCAN*

In [ ]:
def plota_grafico_dispersao(x : pd.Series, y : pd.Series, carteira : str):
    '''
    Constrói gráfico de dispersão relacionando duas colunas de um DataFrame.

    ## Parâmetros
    x (pd.Series): A coluna do eixo X do gráfico.
    
    y (pd.Series): A coluna do eixo Y do gráfico.
    
    carteira (string): O tipo de carteira dos fundos sendo analizados
    no gráfico. Opcional, apenas compõe o título do gráfico.

    ## Retorna
    None.
    '''
    graph = plt.scatter(x, y, c=x)
    if carteira:
        plt.title(f'{carteira}: {x.name} x {y.name}')
    else:
        plt.title(f'{x.name} x {y.name}')
    plt.show()

In [ ]:
def pipeline_dbscan(dados : pd.DataFrame, eps : float, min_samples : int, classificacao : str):
    '''
    Realiza todas as operações relativas ao treinamento dos modelos com o algoritmo DBSCAN.
    Constrói gráfico de dispersão relacionando os clusters e a inadimplência total.

    ## Parâmetros
    dados (pd.DataFrame): O DataFrame a ser usado para cálculo das distâncias
    e do K.

    eps (float): A distância máxima entre dois pontos nos dados para que sejam
    considerados vizinhos.
    
    min_samples (int): O número mínimo de amostras (ou peso total) em um conjunto
    para que um ponto seja considerado um ponto central na clusterização (incluindo
    ele mesmo).
    
    classificacao (string): O tipo de carteira dos fundos sendo analizados
    (presentes no parâmetros "dados").

    ## Retorna
    None.
    '''
    db = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = db.fit_predict(dados.drop(['ID_Participante', 'Data_Competencia', 'Razao_Inadimplencia_Provisao'], axis=1))
    dados['cluster'] = clusters
    plota_grafico_dispersao(dados['cluster'], dados['Razao_Inadimplencia_Provisao'], classificacao)

In [ ]:
# Treino de modelo e plot de gráficos para cada divisão de dados por carteira
for classificacao in dados_classificacao.keys():
    pipeline_dbscan(dados_classificacao[classificacao], 0.5, 20, classificacao)

# Conclusão
A principal diferença entre os resultados gerados pelos algoritmos *DBSCAN* e *Isolation Forest* se refere à clusterização realizada pelo primeiro em comparação à indicação do nível de anormalidade exposto pelo segundo. Ou seja, enquanto o *DBSCAN* agrupa dados que compartilham de características semelhantes em um ou mais *clusters* e isola ruídos, o *Isolation Forest* classifica os dados como anômalos (classificação "-1" do modelo) ou normais (classificação "1" do modelo).

Pelo raciocínio de detectação de anomalias ou ruídos, não é possível avaliar positivamente o desempenho do algoritmo *DBSCAN*, porque, segundo a documentação, os dados anômalos ou ruidosos são alocados no *cluster* "-1", o qual foi criado para poucos tipos de carteira no processo de modelagem e não apresenta muitos resultados relevantes. Por isso, o grupo partiu para a análise geral dos *clusters* gerados, buscando encontrar agrupamentos de fundos problemáticos que não estivessem classificados como "-1" nos diferentes tipos de carteira.

Com o desafio de comparar resultados de diferentes modelos não supervisionados e que ainda possuem processos tão diferentes de funcionamento em vista, criou-se a coluna "Razao_Inadimplencia_Provisao", uma medida que indica o quão maior que a provisão é a inadimplência (quanto maior o valor contido na coluna, maior a inadimplência em relação à provisão), dado que essas são duas *features* centrais e fundamentais para os objetivos do projeto e para análise de fundos problemáticos.

Com a construção de gráficos que demonstram a relação entre essa razão e os *clusters* gerados pelo algoritmo, é possível visualizar que os diferentes *clusters* contêm dados que compartilham certa semelhança no que tange essa característica da razão. Há *outliers* e eles tendem a estar mais presentes em um único *cluster*, porém, em todos os casos, esse mesmo *cluster* também apresenta uma grande quantidade de dados semelhantes aos presentes em outros agrupamentos. Ou seja, mesmo que fossem usados esses grupos de dados aparentemente anômalos, seria difícil filtrar o que realmente seria um fundo em risco, gerando um grande número de "falsos positivos".

Portanto, concluiu-se que o modelo não é o mais apropriado para construção da solução esperada no projeto quando comparado ao desenvolvido com o algoritmo *Isolation Forest*, tanto por sua dificuldade de classificar dados anômalos precisamente com o *dataset* utilizado quanto por trabalhar com clusterização e identificação de padrões, que não é a intenção principal do projeto considerando o caráter caótico dos dados.